# Finetuning using IndoBART

In [1]:
! git clone https://github.com/indobenchmark/indonlg.git

Cloning into 'indonlg'...
remote: Enumerating objects: 152, done.
remote: Counting objects: 100% (152/152), done.
remote: Compressing objects: 100% (113/113), done.
remote: Total 152 (delta 75), reused 97 (delta 34), pack-reused 0
Receiving objects: 100% (152/152), 2.32 MiB | 11.82 MiB/s, done.
Resolving deltas: 100% (75/75), done.


In [2]:
%cd /content/indonlg
! pip install -r requirements.txt

/content/indonlg
     |████████████████████████████████| 1.5 MB 5.5 MB/s 
     |████████████████████████████████| 68 kB 4.8 MB/s 
     |████████████████████████████████| 776.8 MB 17 kB/s 
     |████████████████████████████████| 186 kB 38.3 MB/s 
     |████████████████████████████████| 3.3 MB 21.7 MB/s 
     |████████████████████████████████| 394.7 MB 20 kB/s 
     |████████████████████████████████| 1.2 MB 33.2 MB/s 
     |████████████████████████████████| 22.3 MB 1.7 MB/s 
     |████████████████████████████████| 65 kB 4.0 MB/s 
     |████████████████████████████████| 149 kB 46.5 MB/s 
     |████████████████████████████████| 1.8 MB 33.2 MB/s 
     |████████████████████████████████| 212 kB 46.7 MB/s 
     |████████████████████████████████| 136 kB 45.5 MB/s 
     |████████████████████████████████| 132 kB 45.7 MB/s 
     |████████████████████████████████| 3.8 MB 32.6 MB/s 
     |████████████████████████████████| 14.8 MB 33.1 MB/s 
     |████████████████████████████████| 462 kB 44.8 MB/s 
 

In [1]:
%cd /content/indonlg/examples

/content/indonlg/examples


In [2]:
import os, sys
sys.path.append('../')
os.chdir('../')

import torch
import shutil
import random
import numpy as np
import pandas as pd
from torch import optim
from transformers import MBartForConditionalGeneration

from indobenchmark import IndoNLGTokenizer
from utils.train_eval import train, evaluate
from utils.metrics import generation_metrics_fn
from utils.forward_fn import forward_generation
from utils.data_utils import MachineTranslationDataset, GenerationDataLoader

In [3]:
###
# common functions
###
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    
def count_param(module, trainable=False):
    if trainable:
        return sum(p.numel() for p in module.parameters() if p.requires_grad)
    else:
        return sum(p.numel() for p in module.parameters())
    
# Set random seed
# set_seed(26092020)

# Load Model

In [4]:
bart_model = MBartForConditionalGeneration.from_pretrained('indobenchmark/indobart-v2')
tokenizer = IndoNLGTokenizer.from_pretrained('indobenchmark/indobart-v2')

model = bart_model
model

Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.


MBartForConditionalGeneration(
  (model): MBartModel(
    (shared): Embedding(40004, 768, padding_idx=1)
    (encoder): MBartEncoder(
      (embed_tokens): Embedding(40004, 768, padding_idx=1)
      (embed_positions): MBartLearnedPositionalEmbedding(1026, 768, padding_idx=1)
      (layers): ModuleList(
        (0): MBartEncoderLayer(
          (self_attn): MBartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=Tru

In [5]:
count_param(model)

131543040

# Prepare Dataset

In [6]:
# configs and args

lr = 1e-4
gamma = 0.9
lower = True
step_size = 1
beam_size = 5
max_norm = 10
early_stop = 5

max_seq_len = 512
grad_accumulate = 1
no_special_token = False
swap_source_target = True
model_type = 'indo-bart'
valid_criterion = 'SacreBLEU'

separator_id = 4
speaker_1_id = 5
speaker_2_id = 6

train_batch_size = 8
valid_batch_size = 8
test_batch_size = 8

source_lang = "[indonesian]"
target_lang = "[sundanese]"

optimizer = optim.Adam(model.parameters(), lr=lr)
src_lid = tokenizer.special_tokens_to_ids[source_lang]
tgt_lid = tokenizer.special_tokens_to_ids[target_lang]

model.config.decoder_start_token_id = tgt_lid

# Make sure cuda is deterministic
torch.backends.cudnn.deterministic = True

# create directory
model_dir = './save/MT_SUNIBS_INZNTV/example_id_su'
if not os.path.exists(model_dir):
    os.makedirs(model_dir, exist_ok=True)

device = 'cuda0'
# set a specific cuda device
if "cuda" in device:
    torch.cuda.set_device(int(device[4:]))
    device = "cuda"
    model = model.cuda()

In [7]:
train_dataset_path = './dataset/MT_SUNIBS_INZNTV/train_preprocess.json'
valid_dataset_path = './dataset/MT_SUNIBS_INZNTV/valid_preprocess.json'
test_dataset_path = './dataset/MT_SUNIBS_INZNTV/test_preprocess.json'

train_dataset = MachineTranslationDataset(train_dataset_path, tokenizer, lowercase=lower, no_special_token=no_special_token, 
                                            speaker_1_id=speaker_1_id, speaker_2_id=speaker_2_id, separator_id=separator_id,
                                            max_token_length=max_seq_len, swap_source_target=swap_source_target)
valid_dataset = MachineTranslationDataset(valid_dataset_path, tokenizer, lowercase=lower, no_special_token=no_special_token, 
                                            speaker_1_id=speaker_1_id, speaker_2_id=speaker_2_id, separator_id=separator_id,
                                            max_token_length=max_seq_len, swap_source_target=swap_source_target)
test_dataset = MachineTranslationDataset(test_dataset_path, tokenizer, lowercase=lower, no_special_token=no_special_token, 
                                            speaker_1_id=speaker_1_id, speaker_2_id=speaker_2_id, separator_id=separator_id,
                                            max_token_length=max_seq_len, swap_source_target=swap_source_target)

train_loader = GenerationDataLoader(dataset=train_dataset, model_type=model_type, tokenizer=tokenizer, max_seq_len=max_seq_len, 
                                    batch_size=train_batch_size, src_lid_token_id=src_lid, tgt_lid_token_id=tgt_lid, num_workers=8, shuffle=True)  
valid_loader = GenerationDataLoader(dataset=valid_dataset, model_type=model_type, tokenizer=tokenizer, max_seq_len=max_seq_len, 
                                    batch_size=valid_batch_size, src_lid_token_id=src_lid, tgt_lid_token_id=tgt_lid, num_workers=8, shuffle=False)
test_loader = GenerationDataLoader(dataset=test_dataset, model_type=model_type, tokenizer=tokenizer, max_seq_len=max_seq_len, 
                                   batch_size=test_batch_size, src_lid_token_id=src_lid, tgt_lid_token_id=tgt_lid, num_workers=8, shuffle=False)

# Test model to generate sequences

In [8]:
inputs = ['aku pergi ke toko obat membeli <mask>']
bart_input = tokenizer.prepare_input_for_generation(inputs, return_tensors='pt',
                                         lang_token = '[indonesian]', decoder_lang_token='[indonesian]')

bart_input.to(device)
bart_out = model(**bart_input)
print(tokenizer.decode(bart_input['input_ids'][0]))
print(tokenizer.decode(bart_out.logits.topk(1).indices[:,:].squeeze()))

<s> aku pergi ke toko obat membeli <mask> [indonesian]
<s> aku pergi ke toko obat membeli obat.


In [9]:
inputs = ['aku menyang pasar karo <mask>']
bart_input = tokenizer.prepare_input_for_generation(inputs, return_tensors='pt',
                                         lang_token = '[javanese]', decoder_lang_token='[javanese]')

bart_input.to(device)
bart_out = bart_model(**bart_input)
print(tokenizer.decode(bart_input['input_ids'][0]))
print(tokenizer.decode(bart_out.logits.topk(1).indices[:,:].squeeze()))

<s> aku menyang pasar karo <mask> [javanese]
<s> aku menyang pasar karo tuku </s>


In [10]:
inputs = ['kuring ka pasar senen meuli daging <mask>']
bart_input = tokenizer.prepare_input_for_generation(inputs, return_tensors='pt',
                                         lang_token = '[sundanese]', decoder_lang_token='[sundanese]')

bart_input.to(device)
bart_out = bart_model(**bart_input)
print(tokenizer.decode(bart_input['input_ids'][0]))
print(tokenizer.decode(bart_out.logits.topk(1).indices[:,:].squeeze()))

<s> kuring ka pasar senen meuli daging <mask> [sundanese]
<s> kuring ka pasar senen meuli daging sapi.


# Test model to translate

In [11]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [12]:
test_loss, test_metrics, test_hyp, test_label = evaluate(model, data_loader=test_loader, forward_fn=forward_generation, 
                                                         metrics_fn=generation_metrics_fn, model_type=model_type, 
                                                         tokenizer=tokenizer, beam_size=beam_size, 
                                                         max_seq_len=max_seq_len, is_test=True, 
                                                         device='cuda')

TESTING... : 100%|██████████| 150/150 [04:27<00:00,  1.78s/it]


In [13]:
metrics_scores = []
result_dfs = []

metrics_scores.append(test_metrics)
result_dfs.append(pd.DataFrame({
    'hyp': test_hyp, 
    'label': test_label
}))

result_df = pd.concat(result_dfs)
metric_df = pd.DataFrame.from_records(metrics_scores)

print('== Prediction Result ==')
print(result_df.head())
print()

print('== Model Performance ==')
print(metric_df.describe())

result_df.to_csv(model_dir + "/prediction_result.csv")
metric_df.describe().to_csv(model_dir + "/evaluation_result.csv")

== Prediction Result ==
                                                 hyp  \
0  memang aku tidak sadar akan sesuatu, tetapi bu...   
1                 mereka semua makan sampai kenyang.   
2  kita ini buatan allah, diciptakan dalam kristu...   
3  orang suci semuanya suci ; tetapi bagi orang n...   
4  pergilah petrus dan yohanes kepada teman - t e...   

                                               label  
0  da teu terang naon - na on, tur can tangtu sim...  
1     terus kabeh dala lah ar nepi ka sare ube uhna.  
2  sabab urang mah darma dad am elan allah, anu g...  
3  pikeun anu hatena suci mah, sagala ge suci. sa...  
4  sanggeus leupas, petrus jeung yahya nep angan ...  

== Model Performance ==
           BLEU  SacreBLEU    ROUGE1    ROUGE2    ROUGEL  ROUGELsum
count  1.000000   1.000000  1.000000  1.000000  1.000000   1.000000
mean   1.908154   1.931405  8.151004  1.942852  7.763501   7.758648
std         NaN        NaN       NaN       NaN       NaN        NaN
min    1.908

In [14]:
metric_df

,BLEU,SacreBLEU,ROUGE1,ROUGE2,ROUGEL,ROUGELsum
0,1.908154,1.931405,8.151004,1.942852,7.763501,7.758648


# Fine Tuning & Evaluation

In [15]:
# Train

n_epochs = 10

train(model, train_loader=train_loader, valid_loader=valid_loader, optimizer=optimizer, 
      forward_fn=forward_generation, metrics_fn=generation_metrics_fn, valid_criterion=valid_criterion, 
      tokenizer=tokenizer, n_epochs=n_epochs, evaluate_every=1, early_stop=early_stop, 
      grad_accum=grad_accumulate, step_size=step_size, gamma=gamma, 
      max_norm=max_norm, model_type=model_type, beam_size=beam_size,
      max_seq_len=max_seq_len, model_dir=model_dir, exp_id=0, fp16="", device=device)

(Epoch 1) TRAIN LOSS:3.4355 LR:0.00010000: 100%|██████████| 746/746 [04:50<00:00,  2.57it/s]


(Epoch 1) TRAIN LOSS:3.4355 BLEU:26.08 SacreBLEU:27.85 ROUGE1:44.01 ROUGE2:16.89 ROUGEL:39.65 ROUGELsum:39.64 LR:0.00010000


VALID LOSS:2.9743: 100%|██████████| 100/100 [00:13<00:00,  7.44it/s]


(Epoch 1) VALID LOSS:2.9743 BLEU:18.77 SacreBLEU:18.92 ROUGE1:44.57 ROUGE2:17.24 ROUGEL:40.45 ROUGELsum:40.41


(Epoch 2) TRAIN LOSS:2.5136 LR:0.00009000: 100%|██████████| 746/746 [04:48<00:00,  2.59it/s]


...

(Epoch 9) VALID LOSS:3.3809 BLEU:22.41 SacreBLEU:22.47 ROUGE1:48.65 ROUGE2:21.55 ROUGEL:44.92 ROUGELsum:44.96
count stop: 2


(Epoch 10) TRAIN LOSS:0.2015 LR:0.00003874: 100%|██████████| 746/746 [04:45<00:00,  2.61it/s]


(Epoch 10) TRAIN LOSS:0.2015 BLEU:90.77 SacreBLEU:91.02 ROUGE1:95.69 ROUGE2:91.71 ROUGEL:95.62 ROUGELsum:95.62 LR:0.00003874


VALID LOSS:3.4604: 100%|██████████| 100/100 [00:13<00:00,  7.52it/s]


(Epoch 10) VALID LOSS:3.4604 BLEU:22.79 SacreBLEU:22.88 ROUGE1:48.91 ROUGE2:22.04 ROUGEL:45.28 ROUGELsum:45.27


In [16]:
# Load best model
model.load_state_dict(torch.load(model_dir + "/best_model_0.th"))

<All keys matched successfully>

In [17]:
# Evaluate
test_loss, test_metrics, test_hyp, test_label = evaluate(model, data_loader=test_loader, forward_fn=forward_generation, 
                                                         metrics_fn=generation_metrics_fn, model_type=model_type, 
                                                         tokenizer=tokenizer, beam_size=beam_size, 
                                                         max_seq_len=max_seq_len, is_test=True, 
                                                         device='cuda')

TESTING... : 100%|██████████| 150/150 [04:41<00:00,  1.87s/it]


In [18]:
metrics_scores = []
result_dfs = []

metrics_scores.append(test_metrics)
result_dfs.append(pd.DataFrame({
    'hyp': test_hyp, 
    'label': test_label
}))

result_df = pd.concat(result_dfs)
metric_df = pd.DataFrame.from_records(metrics_scores)

print('== Prediction Result ==')
print(result_df.head())
print()

print('== Model Performance ==')
print(metric_df.describe())

result_df.to_csv(model_dir + "/prediction_result_f.csv")
metric_df.describe().to_csv(model_dir + "/evaluation_result_f.csv")

== Prediction Result ==
                                                 hyp  \
0  memang, simkuring mah henteu sadar kana sagala...   
1         br ak dala lah ar nepi ka sare ube uh eun.   
2  sabab urang teh beunang na ku allah, diciptaka...   
3  jelema - jel ema anu suci, kabeh ge, suci. tap...   
4  sanggeus dib ebas keun, petrus jeung yahya nga...   

                                               label  
0  da teu terang naon - na on, tur can tangtu sim...  
1     terus kabeh dala lah ar nepi ka sare ube uhna.  
2  sabab urang mah darma dad am elan allah, anu g...  
3  pikeun anu hatena suci mah, sagala ge suci. sa...  
4  sanggeus leupas, petrus jeung yahya nep angan ...  

== Model Performance ==
           BLEU  SacreBLEU     ROUGE1     ROUGE2     ROUGEL  ROUGELsum
count   1.00000   1.000000   1.000000   1.000000   1.000000    1.00000
mean   15.14917  15.136601  36.406991  16.484754  31.325453   31.31863
std         NaN        NaN        NaN        NaN        NaN        NaN


In [20]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [21]:
! cp './save/MT_SUNIBS_INZNTV/example_id_su/best_model_0.th' '/content/drive/My Drive/'

In [ ]:
! cp './save/MT_SUNIBS_INZNTV/example_id_su/prediction_result_f.csv' '/content/drive/My Drive/'